In [134]:
# Import pyVPLM modules/functions
from pyvplm.core.definition import PositiveParameter, PositiveParameterSet
from pyvplm.addon.variablepowerlaw import buckingham_theorem

# Import libs
import ipywidgets as widgets
import ipyvuetify as v

Row = 0

def add_item(widget, data, event):
    name = name_entry.v_model
    description = desc_entry.v_model
    unit = unit_entry.v_model
    lower_bound = lb_entry.v_model
    upper_bound = ub_entry.v_model
    name_entry.v_model = ''
    desc_entry.v_model = ''
    unit_entry.v_model = ''
    lb_entry.v_model = None
    ub_entry.v_model = None
    del_but = v.Btn(label = "delete")
    sheet.items = sheet.items + [{"name" : name,
                                  "description" : description,
                                  "unit": unit,
                                  "lower bound" : lower_bound, 
                                  "upper bound" : upper_bound}]

def buckingham(widget, data, event):
    widget.loading = True
    data = sheet.items
    param_list = []
    first = True
    param_set = {}

    for item in data:
        if item['lower bound'] == '':
            bounds = [item['upper bound']]
        else:
            bounds = [item['lower bound'], item['upper bound']]
        param = PositiveParameter(item['name'], bounds, item['unit'], item['description'])
        param_set[item['name']] = param
        if first:
            param_set = PositiveParameterSet(param)
            first = False
    pi_set, _ = buckingham_theorem(param_set, True)
    pi_set.latex_render()
    widget.loading = False
        
    
def del_item(widget, data, event):
    if sheet.v_model:
        item_name = sheet.v_model[0]['name']
        for i in range(len(sheet.items)):
            if sheet.items[i]['name'] == item_name:
                if i == len(sheet.items):
                    sheet.items = sheet.items[:-1]
                else:
                    sheet.items = sheet.items[0:i] + sheet.items[i+1:]
                break
    
def up_item(widget, data, event):
    l = len(sheet.items)
    if l >= 2 and sheet.v_model:
        item_name = sheet.v_model[0]['name']
        for i in range(1, l):
            if sheet.items[i]['name'] == item_name:
                if i == l:
                    sheet.items = sheet.items[0:i-1] + [sheet.items[i]] + [sheet.items[i-1]]
                else:
                    sheet.items = sheet.items[0:i-1] + [sheet.items[i]] + [sheet.items[i-1]] + sheet.items[i+1:]
                break

def down_item(widget, data, event):
    l = len(sheet.items)
    if l >= 2 and sheet.v_model:
        item_name = sheet.v_model[0]['name']
        for i in range(0, l - 1):
            if sheet.items[i]['name'] == item_name:
                if i == l - 1:
                    sheet.items = sheet.items[0:i] + [sheet.items[i+1]] + [sheet.items[i]]
                else:
                    sheet.items = sheet.items[0:i] + [sheet.items[i+1]] + [sheet.items[i]] + sheet.items[i+2:]
                break
    
name_entry = v.TextField(label = "Name", v_model = '', outlined = True)
desc_entry = v.TextField(label = "Description", v_model = '', outlined = True)
unit_entry = v.TextField(label = "Unit", v_model = '', outlined = True)
lb_entry = v.TextField(v_model=None, type="number", label = "Lower bound", outlined = True)
ub_entry = v.TextField(v_model=None, type="number", label = "Upper bound", outlined = True)
add_btn = v.Btn(children = ["Add parameter"], height = 56, width = 305, color="green")
add_btn.on_event('click', add_item)
buck_btn = v.Btn(children = ["Buckingham theorem"], color = "blue", width="50%")
buck_btn.on_event('click', buckingham)

h = [{'text': 'Name', 'sortable': False, 'value': 'name'}, 
    {'text': 'Description','sortable': False, 'value': 'description'},
    {'text': 'Unit', 'sortable': False, 'value': 'unit'},
    {'text': 'Lower bound', 'sortable': False, 'value': 'lower bound'},
    {'text': 'Upper Bound', 'sortable': False, 'value': 'upper bound'}]
it = [{"name" : "x", "description" : "length", "unit": "m", "lower bound" : 0.1, "upper bound" : 100},
    {"name" : "y", "description" : "height", "unit" : "cm", "lower bound" : 1, "upper bound" : 1000},
    {"name" : "t", "description" : "time", "unit" : "s", "lower bound" : 0.5, "upper bound" : 10},
    {"name" : "v", "description" : "speed", "unit" : "m/s", "lower bound" : 1, "upper bound" : 50}]

icon_up = v.Btn(children=[v.Icon(children=["mdi-arrow-up-bold"], large = True)], style_  = "margin : 10px")
icon_down = v.Btn(children=[v.Icon(children=["mdi-arrow-down-bold"], large = True)], style_  = "margin : 10px")
icon_del = v.Btn(children=[v.Icon(children=["mdi-delete"], large = True)], style_  = "margin : 10px")

icon_up.on_event('click', up_item)
icon_down.on_event('click', down_item)
icon_del.on_event('click', del_item)

sheet = v.DataTable(v_model=[{'name' : None}],
                    show_select=True,
                    single_select=True,
                    item_key = 'name',
                    headers=h,
                    items = it,
                    layout = widgets.Layout(flex = '90 1 auto', width = 'auto'))

col1 = v.Col(children=[name_entry,lb_entry])
col2 = v.Col(children=[desc_entry,ub_entry])
col3 = v.Col(children=[unit_entry,add_btn])
box1 = v.Container(children=[v.Row(children=[col1, col2, col3])])

action_box = widgets.VBox([icon_up, icon_down, icon_del])

box2 = widgets.HBox([sheet, action_box])
box2.layout.align_content = "center"
box2.layout.justify_content = "space-between"

box3 = v.Container(children=[v.Row(children=[buck_btn], justify = "center")])

vbox = widgets.VBox([box1, box2, box3])
vbox.layout.margin = "15px 0px 10px 0px"
vbox.layout.border = '2px solid'
vbox